In [2]:
%matplotlib inline
import numpy as np
import os.path
from datetime import date, timedelta
import json
import pandas as pd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
import matplotlib
output_notebook()

Loading BokehJS ...

In [514]:
def calc_total_gold(dft,team,gametime):
    goldTotal = 0
    for item in team:
        ID = str(item['ID'])
        goldTotal += dft['participantFrames'][ID]['totalGold']
    return goldTotal    

In [584]:
def calc_total_kills(dft,gametime):
    kill_data = {}
    kill_data['kills'] = 0
    kill_data['killerId'] = []
    for event in dft['events']:
        if event['eventType'] == 'CHAMPION_KILL':
            kill_data['kills'] += 1  
            kill_data['killerId'].append(event['killerId'])
    return kill_data

In [586]:
#b_type = TOWER_BUILDING or INHIBITOR_BUILDING
def calc_building_kills(dft,gametime,b_type):
    obj_data = {}
    obj_data['kills'] = 0
    obj_data['killerId'] = []
    obj_data['teamId'] = []
    for event in dft['events']:
        if event['eventType'] == 'BUILDING_KILL':
            if event['buildingType'] == b_type:
                obj_data['kills'] += 1
                obj_data['killerId'].append(event['killerId'])
                obj_data['teamId'].append(event['teamId'])   
    return obj_data

In [587]:
#monster = DRAGON, BARON_NASHOR, or RIFTHERALD
def calc_elite_monster_kills(dft,gametime,monster):
    monster_data = {}
    monster_data['kills'] = 0
    monster_data['killerId'] = []
    for event in dft['events']:
        if event['eventType'] == 'ELITE_MONSTER_KILL':
            if event['monsterType'] == monster:
                monster_data['kills'] += 1
                monster_data['killerId'].append(event['killerId'])
    return monster_data

In [580]:
def split_kills(team,killerIds):
    kills = 0
    for item in team:
        ID = item['ID']
        kills += killerIds.count(ID)
    return kills

def split_buildings(team, buildings):
    ID = int(team['teamId'])
    return buildings.count(ID)

In [581]:
matchNum = 2
f = open("./data/real_data/match_%d.json"%matchNum,"r")
data = json.load(f)
game_length = data['timeline']['frames'][-1]['timestamp']/1000/60 
time = 5
df1 = pd.DataFrame(data['participants'])
df2 = pd.DataFrame(data['teams'])

#determine basic stats of winning team
victory_team = None
defeat_team = None
for i in range(2):
    df = df2.iloc[i]
    if df['winner']:
        victory_team = df2.iloc[[i]]
    else:
        defeat_team = df2.iloc[[i]]

In [582]:
#separate winners and losers
winners, losers = [],[]
for i in range(10):
    df = df1.iloc[i]
    item = df['stats']
    playerId = df['participantId']
    rank = df['highestAchievedSeasonTier'] 
    if item['winner']:
        item['ID'] = playerId
        item['rank'] = rank
        item['teamId'] = int(victory_team['teamId'])
        winners.append(item)
    else:
        item['ID'] = playerId
        item['rank'] = rank
        item['teamId'] = int(defeat_team['teamId'])
        losers.append(item)

In [589]:
#go through each match's timestamps/minutes   
timeseries = []
df3 = pd.DataFrame(data['timeline']['frames'])
length = df3.shape[0]
firstBlood,firstTower, firstInhib, firstBaron,firstDragon,firstRift = False, False, False, False, False, False
winFirstBlood,loseFirstBlood = False,False
winFirstTower,loseFirstTower = False, False
winFirstInhib,loseFirstInhib = False, False
winFirstDragon,loseFirstDragon = False, False
winFirstBaron,loseFirstBaron = False, False
winFirstRift,loseFirstRift = False, False
for i in range(1,length):
    df = df3.iloc[i]
    gametime = df['timestamp']/1000/60
    #for each timestamp find: kills, objectives, elitemonsters, gold
    champion = calc_total_kills(df,gametime)
    tower = calc_building_kills(df,gametime,'TOWER_BUILDING')
    inhibitor = calc_building_kills(df,gametime,'INHIBITOR_BUILDING')
    dragon = calc_elite_monster_kills(df,gametime,'DRAGON')
    baron = calc_elite_monster_kills(df,gametime,'BARON_NASHOR')
    riftherald = calc_elite_monster_kills(df,gametime,'RIFTHERALD')
    #print champion
    #find total gold
    winGoldTotal = calc_total_gold(df,winners,gametime)
    loseGoldTotal = calc_total_gold(df,losers,gametime)

    #win and lose kills
    winChampKills = split_kills(winners,champion['killerId'])
    loseChampKills = split_kills(losers,champion['killerId'])

    #win and lose towers/inhibs, careful the teamid that loses a tower is what is defined
    winTowerKills = split_buildings(defeat_team, tower['teamId'])
    loseTowerKills = split_buildings(victory_team,tower['teamId'])

    winInhibKills = split_buildings(defeat_team,inhibitor['teamId'])
    loseInhibKills = split_buildings(victory_team,inhibitor['teamId'])

    #win and lose elite monsters
    winDragonKills = split_kills(winners,dragon['killerId'])
    loseDragonKills = split_kills(losers,dragon['killerId'])

    winBaronKills = split_kills(winners,baron['killerId'])
    loseBaronKills = split_kills(losers,baron['killerId'])

    winRiftKills = split_kills(winners,riftherald['killerId'])
    loseRiftKills = split_kills(losers,riftherald['killerId'])


    #find first bloods,towers,inhibs,dragons,barons,riftheralds
    if not firstBlood and champion['kills'] != 0:
        firstBlood = True
        firstBId = champion['killerId'][0]

        for item in winners:
            if firstBId == item['ID']:
                winFirstBlood = True
                loseFirstBlood = False
                break
            else:
                winFirstBlood = False
                loseFirstBlood = True

    if not firstTower and tower['kills'] != 0:
        firstTower = True
        firstTId = tower['teamId']

        if int(victory_team['teamId']) == firstTId:
            winFirstTower = True
            loseFirstTower = False
        else:
            winFirstTower = False
            loseFirstTower = True


    if not firstInhib and inhibitor['kills'] != 0:
        firstInhib = True
        firstId = inhibitor['teamId']

        if int(victory_team['teamId']) == firstId:
            winFirstInhib = True
            loseFirstInhib = False
        else:
            winFirstInhib = False
            loseFirstInhib = True


    if not firstDragon and dragon['kills'] != 0:
        firstDragon = True
        firstId = dragon['killerId'][0]
        for item in winners:
            if firstId == item['ID']:
                winFirstDragon = True
                loseFirstDragon = False
                break
            else:
                winFirstDragon = False
                loseFirstDragon = True


    if not firstBaron and baron['kills'] != 0:
        firstBaron = True
        firstId = baron['killerId'][0]
        for item in winners:
            if firstId == item['ID']:
                winFirstBaron = True
                loseFirstBaron = False
                break
            else:
                winFirstBaron = False
                loseFirstBaron = True


    if not firstRift and riftherald['kills'] != 0:
        firstRift = True
        firstId = riftherald['killerId'][0]
        for item in winners:
            if firstId == item['ID']:
                winFirstRift = True
                loseFirstRift = False
                break
            else:
                winFirstRift = False
                loseFirstRift = True

    timeseries.append((matchNum,gametime,winGoldTotal,loseGoldTotal,winChampKills,loseChampKills,winTowerKills,loseTowerKills, winInhibKills,loseInhibKills, winDragonKills,loseDragonKills,winBaronKills,loseBaronKills,winRiftKills,loseRiftKills,winFirstBlood,loseFirstBlood, winFirstTower,loseFirstTower,winFirstInhib,loseFirstInhib,winFirstDragon, loseFirstDragon,winFirstBaron,loseFirstBaron,winFirstRift,loseFirstRift))

#make dataframe
match_df = pd.DataFrame(timeseries,columns=['matchId',  
                                            'time', 
                                            'winGoldTotal',
                                            'loseGoldTotal', 
                                            'winChampKills',
                                            'loseChampKills',
                                            'winTowerKills',
                                            'loseTowerKills',
                                            'winInhibKills',
                                            'loseInhibKills',
                                            'winDragonKills',
                                            'loseDragonKills',
                                            'winBaronKills',
                                            'loseBaronKills',
                                            'winRiftKills',
                                            'loseRiftKills',
                                            'winFirstBlood',
                                            'loseFirstBlood',
                                            'winFirstTower',
                                            'loseFirstTower',
                                            'winFirstInhib',
                                            'loseFirstInhib',
                                            'winFirstDragon',
                                            'loseFirstDragon',
                                            'winFirstBaron',
                                            'loseFirstBaron',
                                            'winFirstRift',
                                            'loseFirstRift'])


In [592]:
pd.set_option('display.max_columns',28)
match_df.head(20)

,matchId,time,winGoldTotal,loseGoldTotal,winChampKills,loseChampKills,winTowerKills,loseTowerKills,winInhibKills,loseInhibKills,winDragonKills,loseDragonKills,winBaronKills,loseBaronKills,winRiftKills,loseRiftKills,winFirstBlood,loseFirstBlood,winFirstTower,loseFirstTower,winFirstInhib,loseFirstInhib,winFirstDragon,loseFirstDragon,winFirstBaron,loseFirstBaron,winFirstRift,loseFirstRift
0,2,1,2500,2500,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False
1,2,2,2671,2670,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False
2,2,3,4718,3904,2,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,False,False
3,2,4,6115,5092,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,False,False
4,2,5,7807,6034,1,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,False,False
5,2,6,9021,7552,0,1,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,False,False
6,2,7,10325,9075,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,False,False
7,2,8,11924,9953,1,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,False,False
8,2,9,13716,12079,1,2,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,False,False
9,2,10,14966,13173,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,False,False


In [597]:
df = df3.iloc[0]
if df['events']:
    print 'empty'

empty
